In [89]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Create a toy tool

In [90]:
from langchain.agents import tool

from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for weather online"""
    return f"Searching for {query}"

print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Search for weather online
{'query': {'title': 'Query', 'description': 'Thing to search for', 'type': 'string'}}


In [91]:
search("foo bar asdf")

'Searching for foo bar asdf'

## Create a real tool

### Weather Search

In [92]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
        'temperature_unit': 'fahrenheit',
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°F'

print(get_current_temperature.name)
print(get_current_temperature.description)
print(get_current_temperature.args)

get_current_temperature
get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.
{'latitude': {'title': 'Latitude', 'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}


In [93]:
from langchain.tools.render import format_tool_to_openai_function
temp_function = format_tool_to_openai_function(get_current_temperature)

In [94]:
get_current_temperature({"latitude": 41.875870, "longitude": -72.801400})

'The current temperature is 33.4°F'

### Wikipedia tool

In [95]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

wikipedia_function = format_tool_to_openai_function(search_wikipedia)

In [96]:
search_wikipedia({"query": "Zach Wilson"})

"Page: Zach Wilson\nSummary: Zachary Kapono Wilson (born August 3, 1999) is an American football quarterback for the New York Jets of the National Football League (NFL). He played college football at BYU, where he was a two-time bowl game MVP, and was selected second overall by the Jets in the 2021 NFL Draft.\n\n\n\nPage: 2023 New York Jets season\nSummary: The 2023 season is the New York Jets' 54th season in the National Football League (NFL), their 64th overall, their fifth under general manager Joe Douglas and their third under head coach Robert Saleh. The Jets are looking to improve on their 7–10 record from 2022. After the Seattle Mariners of Major League Baseball (MLB) clinched a playoff berth for the first time since 2001 and the Sacramento Kings of the National Basketball Association (NBA) clinched a playoff berth for first time since 2006, the Jets and the Buffalo Sabres of the National Hockey League (NHL) are now tied for the longest active playoff drought in major North Amer

In [97]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [98]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "title": "Non-oAuth Scopes example",
    "version": "1.0.0"
  },
  "paths": {
    "/users": {
      "get": {
        "security": [
          {
            "bearerAuth": [
              "read:users",
              "public"
            ]
          }
        ]
      }
    }
  },
  "components": {
    "securitySchemes": {
      "bearerAuth": {
        "type": "http",
        "scheme": "bearer",
        "bearerFormat": "jwt",
        "description": "note: non-oauth scopes are not defined at the securityScheme level"
      }
    }
  }
}
"""

In [102]:
spec = OpenAPISpec.from_text(text) #I haven't been able to get this to work
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

AttributeError: 'super' object has no attribute 'parse_obj'

In [ ]:
model.invoke("what are three pets names")

In [ ]:
model.invoke("tell me about pet with id 42")

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [103]:
from langchain.chat_models import ChatOpenAI
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [106]:
model.invoke("what is the weather in SF right now?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})